In [1]:
import torch
import random
from supervoice_hybrid import SupervoceHybridStage1, SentencePieceTextTokenizer
from train.dataset import load_sampler, create_async_loader

/home/steve/miniconda3/envs/gym/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/steve/miniconda3/envs/gym/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


In [2]:
tokenizer = SentencePieceTextTokenizer("./tokenizer_text.model")
train_sampler = load_sampler("./external_datasets/libriheavy/libriheavy_cuts_small.jsonl.gz", "./external_datasets/libriheavy-encodec/", 1, tokenizer)
model = SupervoceHybridStage1().cuda()

In [3]:
audio, text = train_sampler()

/data/notebooks/supervoice-hybrid/train/dataset.py:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoded = torch.load(dir + id + ".pt")


In [4]:
min_duration = 75 * 3
max_duration = audio[0].shape[1] // 2 
if max_duration > min_duration:
    audio_split = random.randint(min_duration, max_duration)
else:
    audio_split = max_duration

text_condition = text[0].cuda()
audio_condition = audio[0][0, :audio_split].cuda()
audio_target = audio[0][0, audio_split:].cuda()
duration = audio[0].shape[1]
print(duration)

1314


In [5]:
model(condition_text = [text_condition, text_condition], condition_audio = [audio_condition, audio_condition], duration=[duration, duration], target = [audio_target, audio_target])

([tensor([[ 0.5083, -0.2311, -0.1917,  ..., -1.2790,  0.6167, -0.4385],
          [ 0.5598, -0.2643,  0.1663,  ..., -0.6868, -0.0769,  0.0515],
          [ 0.1374, -0.5194, -0.2495,  ..., -1.0727,  0.8693,  0.0616],
          ...,
          [ 0.3572, -0.5310,  0.0056,  ..., -0.7215,  0.7556,  0.0211],
          [ 0.1467, -0.6556, -0.0912,  ..., -0.7694,  0.3761, -0.2830],
          [ 0.0910, -0.4948,  0.2226,  ..., -0.8881,  0.6297, -0.3353]],
         device='cuda:0', grad_fn=<SliceBackward0>),
  tensor([[ 0.0122, -0.5449,  0.2809,  ..., -0.7592,  0.5439,  0.1813],
          [-0.3995, -0.0757, -0.1816,  ..., -0.6545,  0.8208, -0.0491],
          [-0.1075, -0.2603,  0.2627,  ..., -0.9913,  0.7549,  0.1915],
          ...,
          [ 0.7413, -0.2365, -0.1692,  ..., -0.2585,  0.9345,  0.1284],
          [ 0.6108, -0.5515, -0.0164,  ..., -0.9712,  0.3632,  0.1100],
          [ 0.7302, -0.3976, -0.5803,  ..., -0.8271,  0.2106,  0.0832]],
         device='cuda:0', grad_fn=<SliceBackward0>)

In [6]:
from xformers.ops import fmha
fmha.BlockDiagonalMask.from_seqlens([duration, duration]).materialize([duration * 2, duration * 2])

tensor([[0., 0., 0.,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        ...,
        [-inf, -inf, -inf,  ..., 0., 0., 0.],
        [-inf, -inf, -inf,  ..., 0., 0., 0.],
        [-inf, -inf, -inf,  ..., 0., 0., 0.]])